In [1]:
import pickle

with open('data/sentiment_comp_qaie_pairs.pkl','rb') as f:
    pairs = pickle.loads(f.read())

In [2]:
print(len(pairs))
pairs = [p for p in pairs if p[1] not in ['无','']]
print(len(pairs))

pairs[0]

12325
12272


['智通财经APP讯，太平洋网络(00543)发布公告，将于2023年6月12日派发截至2022年12月31日止年度的末期股息每股0.1元人民币。\n---\n请从上文中抽取出所有公司/机构、对应的在本文中的情感倾向（积极、消极、中性）以及原因。\n并用这样的格式返回：\n{"ORG":..., "sentiment":..., "reason":...}',
 '{"ORG": "太平洋网络", "sentiment": "积极", "reason": "宣布派发股息每股0.1元人民币"}']

In [6]:
import json
with open('data/sentiment_comp_ie.json','w',encoding='utf8') as f:
    for p in pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

In [3]:
# 打乱顺序重新排列
import json
import random
with open('data/sentiment_comp_ie_shuffled.json','w',encoding='utf8') as f:
    random_pairs = random.sample(pairs,k=len(pairs))
    for p in random_pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

## baichuan-7B inference

In [1]:
# 下载百川大模型看看
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", device_map="auto", trust_remote_code=True).half()

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 尝试从大模型中获取情感标签的概率值

In [36]:
text = "问：今天心情还真不错。这句话的情感是怎样的？答：这句话的情感是："
inputs = tokenizer.encode(text, return_tensors='pt').to('cuda:0')
# outputs = model.generate(inputs,max_new_tokens=50,do_sample=True)
# output = outputs[0][len(inputs[0]):]
# tokenizer.decode(output,skip_special_tokens=True)

In [76]:
import torch
text = "ChatGPT的推出对百度的搜索业务产生的强烈冲击，搜索引擎的作用性在降低"

prompt = f"问：{text}这句话的情感是怎样的？答：这句话的情感是："
inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda:0')

last_logits = model(inputs).logits[:,-1,:]
probs = torch.softmax(last_logits, dim=-1)

targets = {
    0: ['消极','负面','难过'],
    1: ['积极','正面','开心']
}
for key in targets:
    target_probs = []
    for w in targets[key]:
        # 取出非 bos 的第一个token（似乎无法通过 add_special_tokens=False 去掉）
        idx = tokenizer.encode(w)[1]
        current_prob = probs[0, idx].item()
        # print(w, current_prob)
        target_probs.append(current_prob)
    print(key,targets[key],'score(prob*100) =',round(sum(target_probs)*100/len(target_probs),2))
        


0 ['消极', '负面', '难过'] score(prob*100) = 0.2
1 ['积极', '正面', '开心'] score(prob*100) = 0.07


In [71]:
outputs = model.generate(inputs,max_new_tokens=50,do_sample=True)
output = outputs[0][len(inputs[0]):]
tokenizer.decode(output,skip_special_tokens=True)

'不耐烦，极其不耐烦。为什么会不耐烦呢？因为我们总是觉得对方不了解情况。1 先了解一下事情的始末吧！在说这句话的时候，对方可能会说：“事情不是这样子的呀'

In [6]:
model

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "weights/sentiment_comp_ie_shuffled_baichuan-7B")

In [3]:
text = """ChatGPT的提出对谷嘎、万度的搜索业务产生巨大打击，传统搜索引擎的作用性降低了。
与此同时，OChat，Linguo等新兴语义搜索公司，迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐。
腾势、艾里等公司表示会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型。
大型图片供应商视觉中国称ChatGPT对公司业务暂无影响，还在观望状态。
（本文图片来自视觉中国，上观新闻为您报道。）
更多报道：
- 亚牛逊公司关于AIGC的表态
- 巨硬公司昨日在A股上市
---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}"""

In [5]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer(text, return_tensors='pt')
inputs = inputs.to('cuda:0')
output = model.generate(**inputs, max_new_tokens=128,repetition_penalty=1.1, streamer=streamer)
# 模型非常自信！（类似于模型自动纠错的能力）同时幻觉十分严重，哈哈哈（当然，这只是训练了200steps）


1. {"ORG":"百度","SENTIMENT":"+20%", "REASON":"新产品发布/更新;技术突破;行业领先地位提升;市场份额扩大;营收增长;股价上涨"; }
2. {"ORG":"阿里巴巴集团","SENTIMENT":"-50%", "REASON":"竞争对手发展迅猛;业绩下滑;市值缩水;股价下跌"; }
3. {"ORG":"腾讯控股有限公司","SENTIMENT":"+40%", "REASON":"新产品发布


## ChatGLM inference

In [20]:
import torch
from transformers import AutoModel,AutoTokenizer
device=torch.device(2)

model_path = "THUDM/chatglm-6b"
# model_glm = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().to(device)
model_glm = AutoModel.from_pretrained(model_path, trust_remote_code=True,device_map='auto')
tokenizer_glm = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# model_glm

Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.52s/it]


In [ ]:
# load LoRA
model_glm = PeftModel.from_pretrained(model_glm, "weights/sentiment_comp_ie").half()

In [19]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer_glm,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer_glm("我们说的艾里巴巴公司，指的是", return_tensors='pt')
inputs = inputs.to('cuda:2')
output = model_glm.generate(**inputs, max_new_tokens=1024,repetition_penalty=1.1, streamer=streamer)

我不确定您所指的“艾里巴巴公司”是指哪个公司。如果您能提供更多上下文或信息,我将尽力回答您的问题。


In [17]:
print(text)

融资资金将用于完善生成式AI新引擎的构建。

创业邦获悉，自然语言技术AI服务商竹间智能宣布已完成D2轮融资，由金浦投资、金库资本、江苏文投、隽赐资本等联合投资。至今，竹间智能已累计完成7轮融资，陆续引入科沃斯、云晖资本、中银国际等股东机构。值得一提的是，竹间智能已经正式推出运用类ChatGPT技术的成熟AIGC产品，并即将开启新一轮融资。

融资资金将用于完善生成式AI新引擎的构建，把以ChatGPT为代表的大语言模型和AIGC技术全面融入竹间产品体系，结合大小模型构建双引擎驱动产品迭代和技术升级，平衡大小模型的优缺点，全面焕新产品功能和服务模式，并正式将产品同步推向海外市场，成为服务全球企业和用户的跨国NLP能力厂商。

竹间智能由前微软（亚洲）互联网工程院副院长简仁贤于2015年创办，致力于以自然语言处理、情感计算、深度学习、知识工程、文本处理等人工智能技术为基础，将AI能力惠及千行百业。

---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}


tokenizer自动把中文括号转成英文括号了...所以你直接在输出里面可能无法直接匹配到 prompt

## ChatGLM2-6B

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
# model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device='cuda:0')
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device_map='auto')
# model = model.eval()
# model
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)

# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response)

In [ ]:
model.hf_device_map
model.hf_device_map['transformer.output_layer'] = model.hf_device_map['transformer.embedding']
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device_map=model.hf_device_map)
model.hf_device_map

In [21]:
res = tokenizer('你好',return_tensors='pt')
res_glm = tokenizer_glm('你好',return_tensors='pt')

In [ ]:
labels = res['input_ids'].clone()
model(input_ids=res['input_ids'],labels=labels)

# model_glm = model_glm.half()
# labels_glm = res_glm['input_ids'].clone()
# model_glm(input_ids=res_glm['input_ids'],labels=labels_glm)

**对比一下 ChatGLM 跟 ChatGLM2 的结构差别：**

ChatGLM:
```python
ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
```

ChatGLM2:
```python
ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)   # <-- smaller vocab size
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()   # <-- LayerNorm to RMSNorm
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)   # <-- smaller attention out_features
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(                # <-- GLU to MLP
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_features=4096, out_features=65024, bias=False)    # <-- smaller out_features
  )
)
```

In [4]:
model_glm.lm_head

Linear(in_features=4096, out_features=130528, bias=False)

In [10]:
model.transformer.output_layer

Linear(in_features=4096, out_features=65024, bias=False)

In [4]:
print(model.chat(tokenizer,query=text)[0])

{"ORG": "ChatGPT", "sentiment": "消极", "reason": "ChatGPT的提出对谷嘎、万度的搜索业务产生巨大打击，传统搜索引擎的作用性降低了。"},
{"ORG": "OChat", "sentiment": "积极", "reason": "OChat，Linguo等新兴语义搜索公司，迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐。"},
{"ORG": "腾势", "sentiment": "积极", "reason": "腾势、艾里等公司表示会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型。"},
{"ORG": "视觉中国", "sentiment": "中性", "reason": "大型图片供应商视觉中国称ChatGPT对公司业务暂无影响，还在观望状态。"},
{"ORG": "亚牛逊公司", "sentiment": "中性", "reason": "亚牛逊公司关于AIGC的表态中并未提及ChatGPT对公司业务的影响。"},
{"ORG": "巨硬公司", "sentiment": "中性", "reason": "巨硬公司昨日在A股上市与ChatGPT对公司业务的影响无关。"}


In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "weights/sentiment_comp_ie_chatglm2")

In [6]:
print(model.chat(tokenizer,text)[0])

{"ORG": "谷嘎", "sentiment": "消极", "reason": "ChatGPT的提出对其搜索业务产生巨大打击"}
{"ORG": "万度", "sentiment": "消极", "reason": "传统搜索引擎的作用性降低了"}
{"ORG": "OChat", "sentiment": "积极", "reason": "迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐"}
{"ORG": "Linguo", "sentiment": "积极", "reason": "迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐"}
{"ORG": "腾势", "sentiment": "积极", "reason": "会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型"}
{"ORG": "艾里", "sentiment": "积极", "reason": "会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型"}
{"ORG": "视觉中国", "sentiment": "中性", "reason": "称ChatGPT对公司业务暂无影响，还在观望状态"}


# HC3 dataset

In [ ]:
import datasets
from datasets import load_from_disk

hc3 = load_from_disk('data/hc3_chatgpt_qa_all')
hc3

In [2]:
hc3[0]

{'question': '盗贼天赋盗贼怎么加天赋?知道告诉一下下啦~~ ',
 'answer': '如果你在玩角色扮演游戏（RPG），那么你可能是在问如何在游戏中给你的盗贼角色加天赋。具体方法可能因游戏而异，但通常有以下几种方法： \n1. 在游戏开始时选择盗贼天赋：在游戏开始时，你可以选择你想要的天赋。这通常是通过选择不同的角色种族或职业来实现的。 \n2. 在游戏进程中获得天赋：在游戏进程中，你可能会获得一些与盗贼相关的天赋。这可能是通过完成任务、升级或解锁新的技能来实现的。 \n3. 使用道具或装备获得天赋：你可能会发现一些道具或装备，它们可以赋予你一些盗贼天赋。这些道具或装备通常是随机生成的，或者是你在游戏进程中获得的奖励。 \n希望这些信息对你有所帮助！'}

In [3]:
import json
with open('data/hc3_chatgpt_zh_specific_qa.json','w',encoding='utf8') as f:
    for i in range(len(hc3)):
        # line = {'q':hc3[i]['question'], 'a':hc3[i]['answer']}
        line = {'q':'问：'+hc3[i]['question'], 'a':'答：'+hc3[i]['answer']}
        line = json.dumps(line, ensure_ascii=False)
        f.write(line)
        f.write('\n')

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
import torch
device = torch.device('cuda:2')
# model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True).to(device)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", device_map="auto", trust_remote_code=True)



In [58]:
def chat(text):
    streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)

    inputs = tokenizer("问："+text+"答：", return_tensors='pt')
    inputs = inputs.to('cuda:2')
    output = model.generate(**inputs, max_new_tokens=128,
                            do_sample=True,
                            repetition_penalty=1.1, 
                            begin_suppress_tokens=[tokenizer.eos_token_id],
                            # streamer=streamer
                            )
    output = output[0][inputs.input_ids.shape[1]:] # 这样可以防止输出prompt部分
    return tokenizer.decode(output,skip_special_tokens=True)
    # return output
    
# chat('你怎么把断了的螺丝钉钻出来？')
# tokenizer.eos_token_id
# tokenizer.encode('  ')

In [3]:
# model = PeftModel.from_pretrained(model, "weights/rulai_plus_baichuan-7B/")
model = PeftModel.from_pretrained(model, "RLHF/weights/baichaun_rlhf_beyond_chinese_test_6step_40")

In [60]:
output = chat('哎，今天好烦啊')
print(output)

有事情想找老公商量又不好意思？


len = 2500  ~ 9G
len = 5000  ~ 40G

# InternLM

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, get_peft_model

model = AutoModelForCausalLM.from_pretrained('internlm/internlm-chat-7b', trust_remote_code=True, device_map="auto") 
tokenizer = AutoTokenizer.from_pretrained('internlm/internlm-chat-7b', trust_remote_code=True)

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /root/anaconda3/envs/gby did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/root/anaconda3/envs/gby/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path

In [2]:
# 按照惯例，先观察一下模型结构
model

InternLMForCausalLM(
  (model): InternLMModel(
    (embed_tokens): Embedding(103168, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x InternLMDecoderLayer(
        (self_attn): InternLMAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (rotary_emb): InternLMRotaryEmbedding()
        )
        (mlp): InternLMMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): InternLMRMSNorm()
        (post_attention_layernorm): InternLMRMSNorm()
      )
    )
 

In [3]:
import json
from transformers import TextStreamer
def predict(text,instruction):
    streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)
    prompt = instruction + text + '\n输出：'
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = inputs.to('cuda:0')
    output = model.generate(**inputs, max_new_tokens=1024,
                            # do_sample=True,
                            # repetition_penalty=1.1, 
                            temperature=0.8,
                            top_p=0.8,
                            eos_token_id=(2, 103028),
                            # begin_suppress_tokens=[tokenizer.eos_token_id],
                            streamer=streamer
                            )
    output = output[0][inputs.input_ids.shape[1]:] # 这样可以防止输出prompt部分
    return tokenizer.decode(output,skip_special_tokens=True)

text = "首先是环境很舒服，我去过附近不少咖啡馆，像回声，咖啡王子一号店，星巴克等等，感觉他们家的环境绝对是顶尖的之一。\
星巴克适合商务人士喝杯咖啡小憩一会，回声适合安安静静的喝咖啡读书度过一个下午。这里怎么说呢，介于两者之间吧，有咖啡有西餐，\
    在这晃一天都没问题。对于我这单纯学习狗来说，音乐声音有点大，另外每个桌子都有电源，但是mac电脑的电源很不好插。\
        点了一杯香草拿铁，觉得有点甜，不过有免费的柠檬水，可以解渴。图中是芝士玉米圆饼，芝士用料很足，个人感觉不错。\
            有时间的话，在这里喝喝咖啡，读读书度过一个下午真是太舒服了。"

instruction = "给定一段文字，请你进行细粒度方面情感分析，具体包括以下这些方面：交通是否便利,距离商圈远近,是否容易寻找,排队等候时间,服务人员态度,是否容易停车,点菜/上菜速度,价格水平,性价比,折扣力度,装修情况,嘈杂情况,就餐空间,卫生情况,分量,口感,外观,推荐程度,本次消费感受,再次消费的意愿。请抽取出所有方面的情感倾向：正向:1, 中性:0, 负向:-1, 未提及:-2，用json格式返回结果。 评论如下："
output = predict(text, instruction)

# json.loads(output)

{"正向":1,"负向":0,"未提及":0,"中":0,"推荐程度":3,"性价比":3,"折扣力度":0,"装修情况":3,"嘈杂情况":0,"就餐空间":3,"卫生情况":3,"分量":3,"口感":3,"外观":3,"推荐程度":3,"再次消费的意愿":3}<eoa>


In [3]:
lora_path = '../XiaoWen/LLM-Tuning-master/weights/aspect_sentiment_jsonfy_output_10k_plus-internlm-chat-7b'
model = PeftModel.from_pretrained(model, lora_path)


json.loads(predict(text))

{"交通是否便利": "未提及", "距离商圈远近": "未提及", "是否容易寻找": "未提及", "排队等候时间": "未提及", "服务人员态度": "未提及", "是否容易停车": "未提及", "点菜/上菜速度": "未提及", "价格水平": "未提及", "性价比": "未提及", "折扣力度": "未提及", "装修情况": "正面", "嘈杂情况": "中性", "就餐空间": "正面", "卫生情况": "正面", "分量": "未提及", "口感": "中性", "外观": "未提及", "推荐程度": "未提及", "本次消费感受": "正面", "再次消费的意愿": "正面"}


{'交通是否便利': '未提及',
 '距离商圈远近': '未提及',
 '是否容易寻找': '未提及',
 '排队等候时间': '未提及',
 '服务人员态度': '未提及',
 '是否容易停车': '未提及',
 '点菜/上菜速度': '未提及',
 '价格水平': '未提及',
 '性价比': '未提及',
 '折扣力度': '未提及',
 '装修情况': '正面',
 '嘈杂情况': '中性',
 '就餐空间': '正面',
 '卫生情况': '正面',
 '分量': '未提及',
 '口感': '中性',
 '外观': '未提及',
 '推荐程度': '未提及',
 '本次消费感受': '正面',
 '再次消费的意愿': '正面'}

# !!! 重要经验
generate参数的影响巨大，应该跟 modeling_xxx.py 中默认使用的参数对应，能保证比较好的效果！

In [9]:
text = "很喜欢这家的餐，味道正宗，口感很好，是我喜欢的味道，满意值得推荐包装仔细，\
    味道很不错，小哥送的也很快，点赞！味道很惊喜，口味很赞👍相当的满足！"
output_d = json.loads(predict(text))

output_d = {k:v for k,v in output_d.items() if v != '未提及'}
output_d

{"交通是否便利": "未提及", "距离商圈远近": "未提及", "是否容易寻找": "未提及", "排队等候时间": "未提及", "服务人员态度": "未提及", "是否容易停车": "未提及", "点菜/上菜速度": "正面", "价格水平": "未提及", "性价比": "未提及", "折扣力度": "未提及", "装修情况": "未提及", "嘈杂情况": "未提及", "就餐空间": "未提及", "卫生情况": "未提及", "分量": "未提及", "口感": "正面", "外观": "未提及", "推荐程度": "未提及", "本次消费感受": "正面", "再次消费的意愿": "未提及"}


{'点菜/上菜速度': '正面', '口感': '正面', '本次消费感受': '正面'}

In [8]:
with model.disable_adapter(): # 禁用 lora
    output = predict(text)
    try:
        output_d = json.loads(output)
        output_d = {k:v for k,v in output_d.items() if v != '未提及'}
        print(output_d)
    except:
        print(output)


答案：{"pos":0,"neg":1,"neu":0,"text": "喝过最难喝的奶茶，一股水的味道，根本就喝不到奶茶味，全倒了，浪费。芋泥就一点点，珍珠就6小颗。服气了！差评，差评。"}<eoa>

答案：{"pos":0,"neg":1,"neu":0,"text": "喝过最难喝的奶茶，一股水的味道，根本就喝不到奶茶味，全倒了，浪费。芋泥就一点点，珍珠就6小颗。服气了！差评，差评。"}<eoa>
